# Detect place cells

Load the required packages

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import re
import math
import holoviews as hv
import panel as pn
import bisect
hv.extension('bokeh', 'matplotlib')
from IPython.display import display
from ipyfilechooser import FileChooser
import warnings
from scipy.stats import zscore
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from scipy.signal import resample
from scipy.stats import sem
import json
import ipywidgets as widgets
%matplotlib widget
warnings.filterwarnings("ignore")
#%reset
from scipy.interpolate import interp1d
from collections import defaultdict


def remove_outliers_avg_filter(data):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    for i in range(len(data)):
        if not np.isnan(data[i]):  # Skip valid values
            continue
        # Find the closest previous non-NaN value
        prev_idx = i - 1
        while prev_idx >= 0 and np.isnan(data[prev_idx]):
            prev_idx -= 1        
        # Find the closest next non-NaN value
        next_idx = i + 1
        while next_idx < len(data) and np.isnan(data[next_idx]):
            next_idx += 1
        # Compute average if both values exist
        if prev_idx >= 0 and next_idx < len(data):
            filtered_data[i] = (data[prev_idx] + data[next_idx]) / 2
        # If neither exists, NaN remains
    return filtered_data

def find_closest_index_sorted(arr, target):
    idx = bisect.bisect_left(arr, target)  # Find the insertion point
    if idx == 0:
        return 0
    if idx == len(arr):
        return len(arr) - 1
    before = idx - 1
    after = idx
    return before if abs(arr[before] - target) <= abs(arr[after] - target) else after

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

def detect_longest_lowest_sequence(arr, margin=0):
    min_val = np.nanmin(arr)  # Find minimum value
    threshold = min_val + margin  # Define threshold based on margin    
    # Get indices where values are within the threshold
    min_indices = np.where(arr <= threshold)[0]
    # Identify consecutive sequences
    longest_sequence = None
    if len(min_indices) > 0:
        start = min_indices[0]
        max_duration = 0  # Track longest duration        
        for i in range(1, len(min_indices)):
            if min_indices[i] != min_indices[i - 1] + 1:  # Not consecutive
                duration = min_indices[i - 1] - start + 1
                if duration > max_duration:
                    max_duration = duration
                    longest_sequence = (start, min_indices[i - 1], duration)
                start = min_indices[i]  # Reset start index        
        # Check last detected sequence
        duration = min_indices[-1] - start + 1
        if duration > max_duration:
            longest_sequence = (start, min_indices[-1], duration)
    return min_val, threshold, longest_sequence


In [ ]:
cd "C:/Users/Manip2/SCRIPTS/minian/"

In [ ]:
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

In [ ]:
sys.path.append(minian_path)
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

Select the minian folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>", layout=widgets.Layout(width='100%'))
display(fc1)
# Register callback function
fc1.register_callback(update_my_folder)

Import spatial map, Ca2+ traces

In [ ]:
mice=Path(dpath).parent.parent.parent.parent.parent.name
date=Path(dpath).parent.parent.parent.name
sessiontype=Path(dpath).parent.parent.name
hour=Path(dpath).parent.name
print(mice, '-',date, '-', sessiontype ,'-', hour)


minianversion = 'minian'
try: # tries to retrieve minianversion either from a previous run or from a previous notebook
    %store -r minianversion
except:
    print("the minian folder to use was not defined in store")
    minianversion = 'minian' #'minianAB' # or 'minian_intermediate'
    %store minianversion

folderMouse = Path(os.path.join(dpath,minianversion))
print(folderMouse)
minian_ds = open_minian(folderMouse)

StampsMiniscopeFile = Path(os.path.join(dpath, f'timeStamps.csv'))
tsmini=pd.read_csv(StampsMiniscopeFile)['Time Stamp (ms)']
minian_freq=round(1/np.mean(np.diff(np.array(tsmini)/1000)))
print('Miniscope sample rate =', minian_freq, 'Hz')

Ao = minian_ds['A']
Co = minian_ds['C']

try: 
    TodropFile = folderMouse / f'TodropFileAB.json'
    with open(TodropFile, 'r') as f:
        unit_to_drop = json.load(f)
except:
    TodropFile = folderMouse.parent / f'TodropFileAB.json'
    with open(TodropFile, 'r') as f:
        unit_to_drop = json.load(f)
    
C=Co.drop_sel(unit_id=unit_to_drop)
A=Ao.drop_sel(unit_id=unit_to_drop)

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units have been found".format(unit))

Import DeepLabCut data

In [ ]:
# Define parameters
pixel_to_cm = 2.25  
table_center_x, table_center_y = 313, 283  # Center of the cheeseboard table on the video
table_center_x, table_center_y = 300, 270  # Center of the cheeseboard table on the video
table_radius = 290 / 2

# Define functions
def calculate_relative_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance_run(x_coords, y_coords):
    distances = np.sqrt(np.diff(x_coords) ** 2 + np.diff(y_coords) ** 2)
    for i in range(1, len(distances) - 1):
        if np.isnan(distances[i]):
            neighbors = [distances[i-1], distances[i+1]]
            distances[i] = np.mean([x for x in neighbors if not np.isnan(x)])
    total_distance_cm = np.nansum(distances) / pixel_to_cm  # Convert to cm
    return total_distance_cm, distances

def find_long_non_nan_sequences(arr, min_length=100):
    mask = ~np.isnan(arr)  # True for non-NaN values
    diff = np.diff(np.concatenate(([0], mask.astype(int), [0])))  # Add padding to detect edges
    starts = np.where(diff == 1)[0]  # Where a sequence starts
    ends = np.where(diff == -1)[0]   # Where a sequence ends
    sequences = [arr[start:end] for start, end in zip(starts, ends) if (end - start) > min_length]
    return sequences

def remove_outliers_median_filter(data, window=1):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    half_window = window // 2
    for i in range(len(data)):
        # Define window range, ensuring it doesn't exceed bounds
        start = max(0, i - half_window)
        end = min(len(data), i + half_window + 1)
        # Extract local values in window
        local_values = data[start:end]
        # Check if the window contains at least one non-NaN value
        if np.all(np.isnan(local_values)):
            median_value = np.nan  # Keep NaN if no valid numbers
        else:
            median_value = np.nanmedian(local_values)  # Compute median ignoring NaNs
        # Replace only if the current value is not NaN
        if not np.isnan(data[i]):
            filtered_data[i] = median_value
    return filtered_data

def replace_high_speed_points_with_nan(x, y, speed_threshold):
    x = np.array(x, dtype='float')
    y = np.array(y, dtype='float')
    # Compute speed between consecutive points
    dx = np.diff(x)
    dy = np.diff(y)
    speeds = np.sqrt(dx**2 + dy**2)
    # Create mask for speed exceeding threshold
    high_speed_mask = speeds > speed_threshold
    # We mark i+1 as NaN if speed between them is too high
    x_out = x.copy()
    y_out = y.copy()
    for i in range(len(high_speed_mask)):
        if high_speed_mask[i]:
            # Only mark the faster of the two points
            if i > 0 and i < len(x) - 1:
                if speeds[i] > speeds[i - 1]:
                    x_out[i + 1] = np.nan
                    y_out[i + 1] = np.nan
                else:
                    x_out[i] = np.nan
                    y_out[i] = np.nan
    return x_out, y_out

def interpolate_2d_path(x, y, kind='linear', fill='extrapolate'):
    x = np.array(x, dtype='float')
    y = np.array(y, dtype='float')
    indices = np.arange(len(x))
    valid_mask = ~np.isnan(x) & ~np.isnan(y)
    if np.sum(valid_mask) < 2:
        raise ValueError("Not enough valid points to interpolate/extrapolate.")
    interp_x = interp1d(indices[valid_mask], x[valid_mask], kind=kind, fill_value=fill, bounds_error=False)
    interp_y = interp1d(indices[valid_mask], y[valid_mask], kind=kind, fill_value=fill, bounds_error=False)
    x_filled = x.copy()
    y_filled = y.copy()
    nan_mask = np.isnan(x) | np.isnan(y)
    x_filled[nan_mask] = interp_x(indices[nan_mask])
    y_filled[nan_mask] = interp_y(indices[nan_mask])
    return x_filled, y_filled

def limit_speed(x, y, max_speed):
    dx = np.diff(x.copy())
    dy = np.diff(y.copy())
    speeds = np.sqrt(dx**2 + dy**2)
    for i,t in enumerate(speeds):
        if t > max_speed:        
            x[i+1] = x[i] 
            y[i+1] = y[i] 
            x[i+2] = x[i] 
            y[i+2] = y[i] 
    return x, y

def remove_short_sequences(arr, max_len=10):
    arr = np.array(arr, dtype='float')
    result = arr.copy()
    is_value = ~np.isnan(arr)
    i = 0
    while i < len(arr):
        if is_value[i]:
            start = i
            while i < len(arr) and is_value[i]:
                i += 1
            end = i
            seq_len = end - start
            # Check if surrounded by NaNs and short enough
            if seq_len <= max_len:
                left_nan = (start == 0) or np.isnan(arr[start - 1])
                right_nan = (end == len(arr)) or np.isnan(arr[end])  # safe for edge
                if left_nan and right_nan:
                    result[start:end] = np.nan
        else:
            i += 1
    return result

In [ ]:
dlcpath=Path(f'{Path(dpath).parent}/My_First_WebCam/')
for file in os.listdir(dlcpath):
    if file.endswith(('.h5')):
        dlcfile=file
        break
dlc_path = os.path.join(dlcpath, dlcfile)
print(dlcfile)

# Load HDF5 file
df = pd.read_hdf(dlc_path)
directory = os.path.dirname(dlc_path)
timestamps_path = Path(directory,'timeStamps.csv')
if timestamps_path.exists():
    timestamps = pd.read_csv(timestamps_path)
    tswebcam = timestamps['Time Stamp (ms)']
    frame_rate = round(1/(np.mean(np.diff(timestamps.iloc[:,1]))/1000))  # fps
    print(f'Acquisition with DAQ, frame rate = {frame_rate} fps')
else:
    frame_rate = 16  # fps /!\ CHANGE ACCORDING TO YOUR DATA
    print(f'Acquisition with Webcam, frame rate = {frame_rate} fps')

X0 = df.iloc[:, 0]
Y0 = df.iloc[:, 1]

# Remove uncertain location predictions (likelihood < 0.9)
df.iloc[:, 0] = df.apply(lambda row: row.iloc[0] if row.iloc[2] > 0.5 else np.nan, axis=1)
df.iloc[:, 1] = df.apply(lambda row: row.iloc[1] if row.iloc[2] > 0.5 else np.nan, axis=1)

X = df.iloc[:, 0]
Y = df.iloc[:, 1]

# Separate the individual's positions into x and y coordinates
individual_xO= np.array(X.values)
individual_yO = np.array(Y.values)

# Define when the mouse is on the cheeseboard (start)
for i, x in enumerate(individual_xO):
    y = individual_yO[i]
    if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
        individual_xO[i] = np.nan
        individual_yO[i] = np.nan

individual_xOO = remove_short_sequences(individual_xO, max_len=3)
individual_yOO = remove_short_sequences(individual_yO, max_len=3)

x_start = find_long_non_nan_sequences(individual_xOO)[0][0] # first value of the first long non nan sequence
y_start = find_long_non_nan_sequences(individual_yOO)[0][0] # first value of the first long non nan sequence

start_frame = np.where(individual_xOO == x_start)[0][0].item()

individual_xOO[:start_frame]=np.nan # remove any path before the real start
individual_yOO[:start_frame]=np.nan # remove any path before the real start

individual_x1, individual_y1 = replace_high_speed_points_with_nan(individual_xOO, individual_yOO, speed_threshold=10)

#for i in range(len(individual_x1)-1, 0, -1): # Find the last non-NaN value which is not isolated
#    if not np.isnan(individual_x1[i]) and not np.isnan(individual_x1[i-1]):
#        last_frame = i
#        break

last_frame = len(individual_x1)

individual_x2, individual_y2 = interpolate_2d_path(individual_x1[start_frame:last_frame], individual_y1[start_frame:last_frame], kind='nearest')
individual_x3, individual_y3 = limit_speed(individual_x2, individual_y2, max_speed=20)

individual_x = np.concatenate((individual_x1[:start_frame], individual_x3))
individual_y = np.concatenate((individual_y1[:start_frame], individual_y3))

if len(individual_x) == len(tswebcam):
    if timestamps_path.exists():
        start_time = timestamps.iloc[start_frame,1].item() / 1000
        end_time = timestamps.iloc[-1,1].item() / 1000
        duration_trial = end_time - start_time
    else:
        duration_trial = (last_frame - start_frame) / frame_rate
    print(f'Total trial duration: {round(duration_trial)} sec')

    total_distance, speed = calculate_distance_run(individual_x[start_frame:last_frame], individual_y[start_frame:last_frame])
    print(f"Total distance run: {round(total_distance)} cm")
    print(f"Average speed: {round(np.nanmean(speed)/pixel_to_cm*frame_rate,2)} cm/s")

    # Create the plot
    fig, ax = plt.subplots(figsize=(3, 3)) 

    # Plot individual positions over time
    cmap = plt.get_cmap('gnuplot2')
    norm = plt.Normalize(vmin=0, vmax=len(individual_x))

    for i in range(1, len(individual_x)):
        ax.plot(individual_x[i-1:i+1], individual_y[i-1:i+1], color=cmap(norm(i)), linewidth=1)

    #plt.plot(individual_x, individual_y, label="Individual's Path", color='b')

    plt.scatter(x_start, y_start, color='black', s=100, label='Start')
    # Draw cheeseboard circle
    table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
    plt.gca().add_patch(table_circle) 

    # Add labels and title
    ax.set_aspect('equal')
    ax.invert_yaxis()
    plt.title(f'Mouse Path On Cheeseboard Maze')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.legend(loc='upper left')
    plt.show()
else: 
    print(f'Error: Length of DLC data ({len(individual_x)}) does not match length of timestamps ({len(tswebcam)})')

In [ ]:
# Parameters
table_center_x, table_center_y = 313, 283  # Center of the cheeseboard table on the video
table_center_x, table_center_y = 300, 270  # Center of the cheeseboard table on the video
table_radius = 290 / 2
square_size = pixel_to_cm * 6

# Filter out NaNs
valid_mask = ~np.isnan(individual_x) & ~np.isnan(individual_y)
path_x = individual_x[valid_mask]
path_y = individual_y[valid_mask]

# Generate symmetric grid of square centers
n = int(np.floor(2 * table_radius / square_size))
offsets = (np.arange(n) - (n - 1) / 2.0) * square_size
centers_x = table_center_x + offsets
centers_y = table_center_y + offsets

# Count visits per square
counts = defaultdict(int)
for px, py in zip(path_x, path_y):
    if np.sqrt((px - table_center_x)**2 + (py - table_center_y)**2) > table_radius:
        continue  # skip points outside circle
    ix = int(np.floor((px - (table_center_x - n/2 * square_size)) / square_size))
    iy = int(np.floor((py - (table_center_y - n/2 * square_size)) / square_size))
    counts[(ix, iy)] += 1

max_count = max(counts.values())/3 if counts else 1

# Plot
fig, ax = plt.subplots(figsize=(3,3))

# Draw circle outline
theta = np.linspace(0, 2*np.pi, 500)
ax.plot(table_center_x + table_radius*np.cos(theta),
        table_center_y + table_radius*np.sin(theta),
        'k', lw=1)

# Draw squares
for i, cx in enumerate(centers_x):
    for j, cy in enumerate(centers_y):
        if np.sqrt((cx - table_center_x)**2 + (cy - table_center_y)**2) + square_size/np.sqrt(2) <= table_radius:
            count = counts.get((i, j), 0)
            if count > 0:                
                intensity = count / max_count # Map count to viridis colormap
                color = plt.cm.viridis(intensity)
            else:
                color = 'lightgrey'  # no visits
            rect = plt.Rectangle((cx - square_size/2, cy - square_size/2), 
                                 square_size, square_size, 
                                 facecolor=color, edgecolor=None, lw=0.5)
            ax.add_patch(rect)

ax.invert_yaxis()
ax.set_aspect('equal')
plt.show()

Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:
# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name= f"Unit n°", 
    options=[i for i in a.index],
    value=a.index[0]
)

next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

# Define interactivity
@pn.depends(indexes=discrete_slider)
def calciumtrace(indexes):
    index = indexes
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Curve((tsmini/1000, C[position, :]), label=f'Unit n°{index} \nNr #{position}').opts(ylim=(0, 10), xlim=(0, tsmini.values[-1]/1000),frame_height=200, color='red')

@pn.depends(indexes=discrete_slider)
def unitshadow(indexes):
    index = indexes 
    data=A.sel(unit_id=index)
    x = np.linspace(0, 600, 600)
    y = np.linspace(0, 600, 600)
    masked_data = np.where(data < 0.01, np.nan, data) 
    return hv.Image((x, y, masked_data)).opts(cmap='hot', clim=(0, 1))

@pn.depends(indexes=discrete_slider)
def circlepath(indexes):
    index = indexes
    radius = 15
    num_points=100
    theta = np.linspace(0, 2*np.pi, num_points)
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Path((a.iloc[position, 0] + radius * np.cos(theta), a.iloc[position, 1] + radius * np.sin(theta)), group='keep').opts(ylim=(0, 600), xlim=(0, 600), line_color='red', line_width=3) #

In [ ]:
output_size = 120
hv.output(size=int(output_size))

image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(colorbar=False, invert_yaxis=False,cmap="Viridis")

alltraces=hv.NdOverlay({idx: hv.Curve((tsmini/1000, C[idx,:])).opts(frame_height=200, show_legend=False, color='black', alpha=0.2, xlabel='time (s)')
                       for idx in np.arange(len(C))})

start =  hv.VLine(start_time).opts(color='blue', line_width=2)
starttxt = hv.Text(start_time + 5, 9.5, 'Start').opts(text_color='blue')

layout = pn.Column(pn.Row(image * hv.DynamicMap(unitshadow), 
            pn.Column(starttxt * start * hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button),       
                    ),
                    ))   

display(layout)

Choose a neuron to plot

In [ ]:
nr=34

Plot neuron's activity during the trial

In [ ]:
plt.close('all')
plt.figure(figsize=[10,2])
plt.plot(tswebcam[start_frame+1:last_frame]/1000, (speed)/max(speed), 'c', label='Mouse speed')

closest_start= find_closest_index_sorted(tsmini, start_time*1000)
closest_end= find_closest_index_sorted(tsmini, end_time*1000)

Cnr=C[nr,:]

normalized_C = (Cnr - np.min(Cnr)) / (np.max(Cnr.values) - np.min(Cnr.values)) if np.sum(Cnr)!=0 else Cnr
plt.plot(tsmini[:]/1000, normalized_C, 'k', label= f'Neuron #{nr}')

Call=C[:,:]
normalized_Cmean = (np.mean(Call, axis=0) - np.min(np.mean(Call, axis=0))) / (np.max(np.mean(Call, axis=0)) - np.min(np.mean(Call, axis=0))) 
plt.plot(tsmini[:]/1000, normalized_Cmean , 'k', alpha=0.2, label= f'Mean neuron activity')

plt.axvline(x=start_time, color='b', linewidth=2)
plt.text(start_time, plt.gca().get_ylim()[1], 'Start', fontsize=8, color='blue')
plt.xlabel('time (s)')
plt.legend(frameon=False, bbox_to_anchor=(1, 1))
plt.tight_layout() 
plt.show()

Plot neuron on the cheeseboard

In [ ]:
x= individual_x[start_frame:last_frame]
y= individual_y[start_frame:last_frame]
Cnr_ = Cnr[closest_start:closest_end+1].to_numpy()

# Plot circular environment
fig, ax = plt.subplots(figsize=(4, 4))
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Normalize Cnr_ for colormap
norm = mcolors.Normalize(vmin=Cnr_.min(), vmax=Cnr_.max())
cmap = cm.jet

# Plot path with colormap
for i in range(len(x) - 1):
    closest_point= find_closest_index_sorted(tsmini, tswebcam[start_frame+i])
    color = cmap(norm(Cnr_[closest_point]))
    ax.plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=3)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
#plt.scatter(x_start, y_start, color='black', s=100, label='Start')
ax.invert_yaxis()
ax.set_aspect('equal')
plt.title(f'Neuron #{nr}')
plt.tight_layout()
plt.show()

In [ ]:
# Filter out NaNs
valid_mask = ~np.isnan(individual_x) & ~np.isnan(individual_y)
path_x = individual_x[valid_mask]
path_y = individual_y[valid_mask]

x= individual_x[start_frame:last_frame]
y= individual_y[start_frame:last_frame]
Cnr_ = Cnr[closest_start:closest_end+1].to_numpy()

# Generate symmetric grid of square centers
n = int(np.floor(2 * table_radius / square_size))
offsets = (np.arange(n) - (n - 1) / 2.0) * square_size
centers_x = table_center_x + offsets
centers_y = table_center_y + offsets

nr_tot_act_biaised = defaultdict(int) # Neuron activity per square
counts = defaultdict(int) # Count visits per square
for idx, (px, py) in enumerate(zip(x, y)):
    if np.sqrt((px - table_center_x)**2 + (py - table_center_y)**2) > table_radius:
        continue  # skip points outside circle
    ix = int(np.floor((px - (table_center_x - n/2 * square_size)) / square_size))
    iy = int(np.floor((py - (table_center_y - n/2 * square_size)) / square_size))
    closest_point= find_closest_index_sorted(tsmini, tswebcam[start_frame+idx])
    nr_tot_act_biaised[(ix, iy)] += Cnr_[closest_point]
    counts[(ix, iy)] += 1

nr_tot_act = defaultdict(float)
for key in counts.keys():
    if counts[key] != 0:   # avoid division by zero
        nr_tot_act[key] = nr_tot_act_biaised[key] / counts[key]
    else:
        nr_tot_act[key] = np.nan  
max_nr_tot_act = max(nr_tot_act.values())

# Plot
fig, ax = plt.subplots(figsize=(3,3))
# Draw circle outline
theta = np.linspace(0, 2*np.pi, 500)
ax.plot(table_center_x + table_radius*np.cos(theta),
        table_center_y + table_radius*np.sin(theta),
        'k', lw=1)
# Draw squares
for i, cx in enumerate(centers_x):
    for j, cy in enumerate(centers_y):
        if np.sqrt((cx - table_center_x)**2 + (cy - table_center_y)**2) + square_size/np.sqrt(2) <= table_radius:
            nr_act = nr_tot_act.get((i, j), np.nan)
            if ~ np.isnan(nr_act):                
                intensity = nr_act / max_nr_tot_act # Map count to viridis colormap
                color = plt.cm.viridis(intensity)
            else:
                color = 'lightgrey'  # no visits
            rect = plt.Rectangle((cx - square_size/2, cy - square_size/2), 
                                 square_size, square_size, 
                                 facecolor=color, edgecolor='None', lw=0.5)
            ax.add_patch(rect)
ax.invert_yaxis()
ax.set_aspect('equal')
plt.show()

Plot activity of each neuron on the cheeseboard

In [ ]:
nb_subplot=min(C.shape[0], 100)
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(15, 15))
axs = axs.flatten()
plt.tight_layout()

for nr in range(nb_subplot):

    Cds_nr = Cds_all[nr,start_frame:last_frame]
    x= individual_x_nonan[start_frame:last_frame]
    y= individual_y_nonan[start_frame:last_frame]

    # Plot circular environment
    table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
    axs[nr].add_patch(table_circle) 

    # Normalize Cds_nr for colormap
    if Cds_nr.min() != Cds_nr.max():
        norm = mcolors.Normalize(vmin=Cds_nr.min(), vmax=Cds_nr.max())
    else:
        norm = mcolors.Normalize(vmin=0, vmax=1)
    cmap = cm.jet

    # Plot path with colormap
    for i in range(len(x) - 1):
        color = cmap(norm(Cds_nr[i]))
        axs[nr].plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=3)

    # Add colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # Scatter start and reward points
    axs[nr].scatter(x_start, y_start,  marker='+', color='black', s=100, label='Start')

    # Customize plot
    axs[nr].invert_yaxis()
    axs[nr].set_aspect('equal')
    axs[nr].set_title(f'#{nr}', pad=0, loc='left')
    
    # Remove box (spines)
    for spine in axs[nr].spines.values():
        spine.set_visible(False)
    axs[nr].set_xticks([])  # No x-axis ticks
    axs[nr].set_yticks([])  # No y-axis ticks

# Adjust layout to avoid clipping
fig.subplots_adjust(wspace=0, hspace=0)
plt.show()

Average neuron activity on the cheeseboard

In [ ]:
Cds_all_mean=np.mean(Cds_all[:,start_frame:last_frame], axis=0)
x= individual_x_nonan[start_frame:last_frame]
y= individual_y_nonan[start_frame:last_frame]

# Plot circular environment
fig, ax = plt.subplots(figsize=(4, 4))
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Normalize Cds_all_mean for colormap
norm = mcolors.Normalize(vmin=Cds_all_mean.min(), vmax=Cds_all_mean.max())
cmap = cm.jet

# Plot path with colormap
for i in range(len(x) - 1):
    color = cmap(norm(Cds_all_mean[i]))
    ax.plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=3)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
plt.scatter(x_start, y_start, color='black', s=100, label='Start')
ax.invert_yaxis()
ax.set_aspect('equal')
plt.title(f'Average activity')
plt.tight_layout()
plt.show()

# Head direction cells

Head orientation cell

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import xarray as xr
import json
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D  # nécessaire pour 3D même si pas utilisé explicitement

# ======================== Paramètres ========================
nr = 5  # Numéro du neurone à visualiser (1-based)
dpath = Path(r"S:\forgetting\Clementine\CheeseboardExperiment\Juin\YL\Cheeseboard\2025_06_05\11_19_07\My_V4_Miniscope")
minianversion = "minian"

# ======================== Définition des chemins ========================
folderMouse = dpath / minianversion
StampsMiniscopeFile = dpath / "timeStamps.csv"
A_zarr_path = folderMouse / "A.zarr"
C_zarr_path = folderMouse / "C.zarr"
todrop_path = dpath / "TodropFileAB.json"
head_orientation_file = dpath / "headOrientation.csv"

# ======================== Vérification des fichiers ========================
for path in [dpath, folderMouse, A_zarr_path, C_zarr_path, StampsMiniscopeFile, todrop_path, head_orientation_file]:
    if not path.exists():
        raise FileNotFoundError(f"Fichier manquant : {path}")

# ======================== Chargement des données calcium ========================
A_ds = xr.open_zarr(A_zarr_path)
C_ds = xr.open_zarr(C_zarr_path)

# ======================== Chargement des timestamps ========================
tsmini = pd.read_csv(StampsMiniscopeFile)['Time Stamp (ms)']
minian_freq = round(1 / np.mean(np.diff(np.array(tsmini) / 1000)))
print('🎞️  Fréquence d\'échantillonnage miniscope =', minian_freq, 'Hz')

# ======================== Application du filtre Todrop ========================
with open(todrop_path, 'r') as f:
    todrop_data = json.load(f)
to_drop = todrop_data
print(f"🧹 Unités à exclure : {len(to_drop)}")

all_units = np.arange(C_ds['C'].shape[0])
kept_units = np.setdiff1d(all_units, to_drop)
print(f"✅ Unités conservées : {len(kept_units)}")

if nr < 1 or nr > len(kept_units):
    raise ValueError(f"nr doit être entre 1 et {len(kept_units)} après filtrage")

C_filtered = C_ds['C'].values[kept_units]
A_filtered = A_ds['A'].values[:, kept_units]
Cds_all = C_filtered

# ======================== Chargement et conversion des données d'orientation ========================
head_df = pd.read_csv(head_orientation_file)

# Extraction des quaternions
qw = head_df['qw'].to_numpy()
qx = head_df['qx'].to_numpy()
qy = head_df['qy'].to_numpy()
qz = head_df['qz'].to_numpy()

# ======================== Conversion des quaternions en angles d'Euler ========================
def quaternion_to_euler(qw, qx, qy, qz):
    sinr_cosp = 2 * (qw * qx + qy * qz)
    cosr_cosp = 1 - 2 * (qx * qx + qy * qy)
    roll = np.arctan2(sinr_cosp, cosr_cosp)

    sinp = 2 * (qw * qy - qz * qx)
    sinp = np.clip(sinp, -1.0, 1.0)
    pitch = np.arcsin(sinp)

    siny_cosp = 2 * (qw * qz + qx * qy)
    cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
    yaw = np.arctan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

roll, pitch, yaw = quaternion_to_euler(qw, qx, qy, qz)
roll_deg = np.degrees(roll)
pitch_deg = np.degrees(pitch)
yaw_deg = np.degrees(yaw)

# ======================== Filtrage des outliers ========================
def remove_outliers_iqr(data, factor=1.5):
    q1 = np.nanpercentile(data, 25)
    q3 = np.nanpercentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr

    data_filtered = data.copy()
    data_filtered[(data < lower_bound) | (data > upper_bound)] = np.nan
    return data_filtered

roll_deg = remove_outliers_iqr(roll_deg)
pitch_deg = remove_outliers_iqr(pitch_deg)
yaw_deg = remove_outliers_iqr(yaw_deg)

valid_mask = ~(np.isnan(roll_deg) | np.isnan(pitch_deg) | np.isnan(yaw_deg))
valid_indices = np.where(valid_mask)[0]

if len(valid_indices) == 0:
    raise ValueError("Aucune donnée d'orientation valide trouvée")

start_frame = valid_indices[0]
last_frame = valid_indices[-1] + 1


# Extraction des données valides
individual_roll = roll_deg[start_frame:last_frame]
individual_pitch = pitch_deg[start_frame:last_frame]
individual_yaw = yaw_deg[start_frame:last_frame]

# Alignement avec données calcium
if start_frame >= Cds_all.shape[1] or last_frame > Cds_all.shape[1]:
    last_frame = min(last_frame, Cds_all.shape[1])
    individual_roll = roll_deg[start_frame:last_frame]
    individual_pitch = pitch_deg[start_frame:last_frame]
    individual_yaw = yaw_deg[start_frame:last_frame]

Cds_nr = Cds_all[nr - 1, start_frame:last_frame]

# ======================== Visualisation 3D interactive ========================

def plot_head_orientation_neuron_3D(nr):
    if nr < 1 or nr > len(kept_units):
        print(f"nr doit être entre 1 et {len(kept_units)}")
        return

    Cds_nr = Cds_all[nr - 1, start_frame:last_frame]
    valid = ~(np.isnan(individual_roll) | np.isnan(individual_pitch) | np.isnan(individual_yaw))

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    norm = mcolors.Normalize(vmin=Cds_nr.min(), vmax=Cds_nr.max())
    cmap = cm.jet

    sc = ax.scatter(individual_roll[valid], individual_pitch[valid], individual_yaw[valid],
                    c=Cds_nr[valid], cmap=cmap, norm=norm, s=10, alpha=0.8)
    ax.set_xlabel("Roll (°)")
    ax.set_ylabel("Pitch (°)")
    ax.set_zlabel("Yaw (°)")
    ax.set_title(f"3D Head Orientation - Neuron #{nr}")

    cbar = fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Calcium activity")

    plt.show()

# Widget interactif pour choisir le neurone
print("🎮 Visualisation interactive 3D : Choisissez un neurone")
slider_neuron = widgets.IntSlider(value=nr, min=1, max=len(kept_units), step=1, description='Neuron:')
widgets.interact(plot_head_orientation_neuron_3D, nr=slider_neuron)




In [ ]:
#Neurones Choisis
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import json
from mpl_toolkits.mplot3d import Axes3D  # required for 3D plots

# === Step 1: Load kept_units from JSON file if available ===
try:
    with open("TodropFileAB.json", "r") as f:
        todrop = json.load(f)
    kept_units = [i for i in range(Cds_all.shape[0]) if i not in todrop["to_drop"]]
    print(f"✅ {len(kept_units)} units kept after filtering.")
except FileNotFoundError:
    try:
        kept_units = list(range(Cds_all.shape[0]))
        print(f"✅ {len(kept_units)} units used without filtering.")
    except NameError:
        kept_units = []

# === Step 2: Visualization function ===
def plot_head_orientation_neurons_grid_selected(point_size=3, alpha=0.7):
    if not kept_units:
        print("❌ No available units to plot. Make sure 'Cds_all' is loaded.")
        return

    neuron_indices = [101, 107, 110, 114]  # 1-based indices in kept_units
    neuron_indices = [nr for nr in neuron_indices if 1 <= nr <= len(kept_units)]

    if not neuron_indices:
        print("❌ No valid neurons to display.")
        return

    n_neurons = len(neuron_indices)
    n_cols = 2
    n_rows = (n_neurons + n_cols - 1) // n_cols

    fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols,
                            figsize=(6 * n_cols, 5 * n_rows),
                            subplot_kw={'projection': '3d'})
    
    axs = np.array(axs).flatten() if n_neurons > 1 else np.array([axs])
    all_colors = []

    for i, nr in enumerate(neuron_indices):
        index_data = kept_units[nr - 1]
        Cds_nr = Cds_all[index_data, start_frame:last_frame]

        valid_mask = ~np.isnan(individual_roll) & ~np.isnan(individual_pitch) & ~np.isnan(individual_yaw)
        if np.sum(valid_mask) == 0:
            print(f"❌ Neuron #{nr}: no valid data.")
            continue

        x_data = individual_yaw[valid_mask]
        y_data = individual_roll[valid_mask]
        z_data = individual_pitch[valid_mask]
        colors = Cds_nr[valid_mask]
        all_colors.append(colors)

        ax = axs[i]
        ax.scatter(x_data, y_data, z_data,
                   c=colors, cmap=cm.jet,
                   s=point_size, alpha=alpha,
                   edgecolors='none')

        ax.set_title(f"Neuron #{nr}", fontsize=11)
        ax.set_xlabel("Yaw (left ↔ right)", fontsize=10)
        ax.set_ylabel("Roll (side tilt)", fontsize=10)
        ax.set_zlabel("Pitch (up ↕ down)", fontsize=10)
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    for j in range(i + 1, len(axs)):
        fig.delaxes(axs[j])

    if all_colors:
        all_colors_concat = np.concatenate(all_colors)
        norm = mcolors.Normalize(vmin=all_colors_concat.min(), vmax=all_colors_concat.max())
        sm = cm.ScalarMappable(cmap=cm.jet, norm=norm)
        sm.set_array([])

        cbar_ax = fig.add_axes([0.25, 0.07, 0.5, 0.02])
        cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
        cbar.set_label('Calcium activity (a.u.)', fontsize=12)

    fig.suptitle("🧠 3D Head Orientation – Neurons #101, 107, 110, 114", fontsize=16, fontweight='bold')
    plt.subplots_adjust(top=0.88, bottom=0.15)
    plt.show()

# === Call the function ===
plot_head_orientation_neurons_grid_selected()



In [ ]:
#STATS
#The three statistical tests used here—permutation, Kolmogorov-Smirnov, and Fisher tests—are
#designed to identify place cells and head direction cells by comparing the observed spatial firing
#patterns of neurons against null distributions that assume no spatial selectivity. These tests suppose
#that true place or head direction cells will show significantly non-random spatial firing patterns that
#cannot be explained by chance alone, with the permutation test shuffling temporal relationships to
#test spatial specificity, the Kolmogorov-Smirnov test comparing the distribution of firing rates
#across spatial bins to a uniform distribution, and the Fisher test evaluating the statistical significance
#of the spatial information content of each neuron's firing pattern.
#Here we used alpha = 10%
import os
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import json
from scipy.stats import ks_2samp, fisher_exact
from IPython.display import display

# ======== PARAMÈTRES ========
dpath = Path(r"S:\forgetting\Clementine\CheeseboardExperiment\Juin\YL\Cheeseboard\2025_06_05\11_19_07\My_V4_Miniscope")
minianversion = "minian"

folderMouse = dpath / minianversion
A_zarr_path = folderMouse / "A.zarr"
C_zarr_path = folderMouse / "C.zarr"
todrop_path = dpath / "TodropFileAB.json"
head_orientation_file = dpath / "headOrientation.csv"

# ======== Vérification des fichiers ========
for path in [dpath, folderMouse, A_zarr_path, C_zarr_path, todrop_path, head_orientation_file]:
    if not path.exists():
        raise FileNotFoundError(f"Fichier manquant : {path}")

print("✅ Tous les fichiers requis existent")

# ======== Chargement données calcium ========
A_ds = xr.open_zarr(A_zarr_path)
C_ds = xr.open_zarr(C_zarr_path)

with open(todrop_path, 'r') as f:
    to_drop = json.load(f)

all_units = np.arange(C_ds['C'].shape[0])
kept_units = np.setdiff1d(all_units, to_drop)
C_filtered = C_ds['C'].values[kept_units]

# ======== Chargement et conversion orientation tête (quaternion → angles) ========
head_df = pd.read_csv(head_orientation_file)

qw = head_df['qw'].to_numpy()
qx = head_df['qx'].to_numpy()
qy = head_df['qy'].to_numpy()
qz = head_df['qz'].to_numpy()

def quaternion_to_euler(qw, qx, qy, qz):
    sinr_cosp = 2 * (qw * qx + qy * qz)
    cosr_cosp = 1 - 2 * (qx * qx + qy * qy)
    roll = np.arctan2(sinr_cosp, cosr_cosp)

    sinp = 2 * (qw * qy - qz * qx)
    sinp = np.clip(sinp, -1.0, 1.0)
    pitch = np.arcsin(sinp)

    siny_cosp = 2 * (qw * qz + qx * qy)
    cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
    yaw = np.arctan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

roll, pitch, yaw = quaternion_to_euler(qw, qx, qy, qz)
roll_deg = np.degrees(roll)
pitch_deg = np.degrees(pitch)
yaw_deg = np.degrees(yaw)

def remove_outliers_iqr(data, factor=1.5):
    q1 = np.nanpercentile(data, 25)
    q3 = np.nanpercentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr
    filtered = data.copy()
    filtered[(data < lower_bound) | (data > upper_bound)] = np.nan
    return filtered

roll_deg = remove_outliers_iqr(roll_deg)
pitch_deg = remove_outliers_iqr(pitch_deg)
yaw_deg = remove_outliers_iqr(yaw_deg)

valid_mask = ~(np.isnan(roll_deg) | np.isnan(pitch_deg) | np.isnan(yaw_deg))
valid_indices = np.where(valid_mask)[0]

if len(valid_indices) == 0:
    raise ValueError("Aucune donnée d'orientation valide")

start_frame = valid_indices[0]
last_frame = valid_indices[-1] + 1

roll_deg = roll_deg[start_frame:last_frame]
pitch_deg = pitch_deg[start_frame:last_frame]
yaw_deg = yaw_deg[start_frame:last_frame]

if last_frame > C_filtered.shape[1]:
    last_frame = C_filtered.shape[1]

C_filtered = C_filtered[:, start_frame:last_frame]

# ======== Fonctions de test de modulation activité vs angle tête ========

def compute_spatial_info(activity, angle, nbins=10):
    bins = np.linspace(np.nanmin(angle), np.nanmax(angle), nbins + 1)
    inds = np.digitize(angle, bins) - 1
    inds[inds == nbins] = nbins - 1

    p_i = np.array([np.sum(inds == i) for i in range(nbins)]) / len(angle)
    r_i = np.array([np.nanmean(activity[inds == i]) for i in range(nbins)])
    r_i = np.nan_to_num(r_i)
    r = np.nanmean(activity)

    with np.errstate(divide='ignore', invalid='ignore'):
        info_per_bin = p_i * (r_i / r) * np.log2((r_i / r) + 1e-12)
    info_per_bin[np.isnan(info_per_bin)] = 0

    return np.nansum(info_per_bin)

def permutation_test(activity, angle, n_perm=1000, nbins=10):
    info_real = compute_spatial_info(activity, angle, nbins=nbins)
    info_perm = np.zeros(n_perm)
    n = len(angle)
    for i in range(n_perm):
        shift = np.random.randint(n)
        activity_perm = np.roll(activity, shift)
        info_perm[i] = compute_spatial_info(activity_perm, angle, nbins=nbins)
    p_val = np.sum(info_perm >= info_real) / n_perm
    return info_real, info_perm, p_val

# ======== Analyse ========

angle_to_use = yaw_deg  # Choisir ici roll_deg ou pitch_deg si souhaité
nbins = 10
percentile_threshold = 90

results = []

for idx, neuron in enumerate(kept_units):
    activity = C_filtered[idx, :]
    valid_idx = ~np.isnan(angle_to_use)
    angle_vals = angle_to_use[valid_idx]
    activity_vals = activity[valid_idx]

    if len(activity_vals) < 10:
        continue

    # Test permutation
    info_real, info_perm, p_perm = permutation_test(activity_vals, angle_vals, n_perm=1000, nbins=nbins)
    threshold = np.percentile(info_perm, percentile_threshold)
    is_modulated_perm = info_real > threshold

    # Test KS (différence distribution activité / bins angle)
    bins = np.digitize(angle_vals, bins=np.linspace(np.nanmin(angle_vals), np.nanmax(angle_vals), nbins + 1)) - 1
    binned_means = np.array([np.nanmean(activity_vals[bins == i]) for i in range(nbins)])
    binned_means = np.nan_to_num(binned_means)
    ks_stat, ks_p = ks_2samp(activity_vals, binned_means)
    is_modulated_ks = ks_p < 0.05

    # Test Fisher (association activité forte vs bins)
    threshold_act = np.nanpercentile(activity_vals, 80)
    active = activity_vals > threshold_act
    spatial_bin = bins < (len(bins) // 2)
    contingency = pd.crosstab(active, spatial_bin)
    if contingency.shape == (2, 2):
        _, fisher_p = fisher_exact(contingency)
    else:
        fisher_p = np.nan
    is_modulated_fisher = fisher_p < 0.05 if not np.isnan(fisher_p) else False

    # Consensus (au moins 2 tests positifs)
    test_flags = [is_modulated_perm, is_modulated_ks, is_modulated_fisher]
    consensus = sum(test_flags) >= 2

    results.append({
        "Neuron": neuron,
        "Info_Spatial (Permutation)": info_real,
        f"Permutation_Threshold_{percentile_threshold}%": threshold,
        "P_Value_Permutation": p_perm,
        "Modulated_Permutation": is_modulated_perm,
        "KS_Stat": ks_stat,
        "P_Value_KS": ks_p,
        "Modulated_KS": is_modulated_ks,
        "P_Value_Fisher": fisher_p,
        "Modulated_Fisher": is_modulated_fisher,
        "Consensus_Modulated": consensus
    })

df_results = pd.DataFrame(results)
df_results.to_csv(dpath / "head_orientation_cells_results.csv", index=False)

print(f"✅ Analyse terminée, résultats sauvegardés dans {dpath / 'head_orientation_cells_results.csv'}")

display(
    df_results[df_results["Consensus_Modulated"] == True]
    .style
    .format(precision=4)
    .set_caption("Résultats détection head orientation cells - Consensus = True")
    .background_gradient(cmap="YlGnBu", subset=[
        "Info_Spatial (Permutation)", "P_Value_Permutation", "P_Value_KS", "P_Value_Fisher"
    ])
)


